### Yelp Reviews: Authorship Attribution with Python and scikit-learn

When people write text, they do so in their own specific style. Often, it’s possible to identify someone using only their unique style of writing. In this post, we’ll see how easy it is to identify people using their writing style through machine learning. Specifically, we’ll look at reviewers who have left multiple reviews on Yelp. We’ll teach a machine learning system to differentiate between different writing styles, and then see how well it can predict the correct author of a review, looking only at the review text.

### OVERVIEW

In this notebook, we will:

Load about 6 million reviews from the 2018 Yelp Dataset Challenge

Find users who have left at least 500 reviews

Train a support vector machine classifier to identify the writing style of each author

See how well the classifier can identify reviews it hasn’t seen during training

Relying on scikit-learn for the machine learning components.

### Yelp Dataset JSON Download and Documentation

https://www.yelp.com/dataset/download

https://www.yelp.com/dataset/documentation/main

In [1]:
from collections import Counter

import json

#### review.json
Contains full review text data including the user_id that wrote the review and the business_id the review is written for.

In [2]:
json_review_dataset = "yelp_dataset/yelp_academic_dataset_review.json"

In [3]:
%%time
reviews = [json.loads(review) for review in open(json_review_dataset)]

CPU times: user 55.7 s, sys: 8.31 s, total: 1min 4s
Wall time: 1min 4s


We’ll take a look at the first review so we know how the data is structured.

In [4]:
print(f"Number of reviews: {len(reviews)}")
print()
print("Looking at the first review to learn how the data is structured.")
reviews[0]

Number of reviews: 5996996

Looking at the first review to learn how the data is structured.


{'review_id': 'x7mDIiDB3jEiPGPHOmDzyw',
 'user_id': 'msQe1u7Z_XuqjGoqhB0J5g',
 'business_id': 'iCQpiavjjPzJ5_3gPD5Ebg',
 'stars': 2,
 'date': '2011-02-25',
 'text': "The pizza was okay. Not the best I've had. I prefer Biaggio's on Flamingo / Fort Apache. The chef there can make a MUCH better NY style pizza. The pizzeria @ Cosmo was over priced for the quality and lack of personality in the food. Biaggio's is a much better pick if youre going for italian - family owned, home made recipes, people that actually CARE if you like their food. You dont get that at a pizzeria in a casino. I dont care what you say...",
 'useful': 0,
 'funny': 0,
 'cool': 0}

### GETTING THE TOP REVIEWERS

In [ ]:
star_rating = Counter([review['stars'] for review in reviews]).most_common(5)

In [6]:
%%time
prolific_reviewers = Counter([review['user_id'] for review in reviews]).most_common(500)

CPU times: user 2.01 s, sys: 95.2 ms, total: 2.11 s
Wall time: 2.1 s


In [ ]:
most_rated_businesses = Counter([review['business_id'] for review in reviews]).most_common(100)

In [ ]:
useful = Counter([review['useful'] for review in reviews]).most_common(100)

In [ ]:
funny = Counter([review['funny'] for review in reviews]).most_common(100)

In [ ]:
cool = Counter([review['cool'] for review in reviews]).most_common(100)

In [ ]:
review_length = Counter([len(review['text']) for review in reviews]).most_common(100)

In [ ]:
prolific_reviewers

Now we want to create a balanced dataset — i.e., we want the same number of reviews of each reviewer. We’ll go through all our reviews again and keep only those reviews written by the 100 authors we identified above, and only 500 reviews from each author. Below, keep_ids is a dictionary which we’ll use to keep count of how many reviews we have from each author.

In [7]:
keep_ids = {pr[0] : 0 for pr in prolific_reviewers}
# keep_ids

In [8]:
keep_reviews = []
for review in reviews:
    uid = review['user_id']
    if uid in keep_ids and keep_ids[uid] < 500:
        keep_reviews.append(review)
        keep_ids[uid] += 1

In [9]:
keep_reviews[0]

{'review_id': '1TIlOJFKldnbl2hyTC7D1g',
 'user_id': '71XStF5U0kmJw2Iym9fclQ',
 'business_id': 'll3Fy_zjbMJ5R6tokibSYg',
 'stars': 4,
 'date': '2014-02-02',
 'text': "I have no complaints about this clinic. The RPharm was knowledgeable, and we had no trouble getting an appointment after work. Both times we've been, they had stock of all the vaccines we needed, e.g. Thus time to go to the Amazon jungle. They also sell accessories like bed nets, carry-on sized deet, etc.. Only inconvenience is that they don't do direct billing for insurance.",
 'useful': 4,
 'funny': 2,
 'cool': 1}

In [10]:
len(keep_reviews)

197302

Now we’ll split the reviews we kept into two lists: one for the texts of the reviews, and another for the author ids. The two lists are implicitly associated by index (i.e., the first text in our texts array was written by the first author in our authors array). In machine learning, we refer to these as “instances” and “labels”. Instances are the things we use to learn from, and the labels are the things we are trying to learn.

In [ ]:
texts = [review['text'] for review in keep_reviews]
authors = [review['user_id'] for review in keep_reviews]

Next, we need to import some things from the scikit-learn library. Specifically, we need a vectorizer (something that transforms our texts into a numerical representation that’s easier to work with) and a classifier (the thing that learns how to discriminate based on labeled examples). We’ll be using TfidfVectorizer, which transforms our text into vectors with tf-idf weighting and a LinearSVM, which is a Support Vector Machine with a linear kernel — a kernel that is often used for text classification tasks. We’ll also import a helper function called train_test_split. We’ll use this to split our data into a training set and a test set. The classifier will learn patterns from the training set, and then we’ll make sure that it actually works by seeing if it can correctly predict the authors in the held-out test set.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split

Now we can transform our texts into vectors by setting up a vectorizer and giving it the list of our texts.

In [ ]:
%%time
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(texts)
vectors.shape

Our vectors variable contains a sparse matrix with shape 50000 (which is 500 texts times by 100 authors — the number of texts we kept) by 68619 which is the number of features. 

Our features are single words (unigrams) and each text is represented by an indication of how often that word appears in that text (which is nearly always 0 as we have 68619 unique words in the dataset).

Often in machine learning, you’ll see the instances (texts in our case) referred to as Xs and the labels as ys. You can think about machine learning tasks as a function y = f(x). We have x (the review text) and we want to know y (the author’s ID). The SVM attempts to learn a function f that can map the texts to the labels. We’ll follow this convention, and break our texts into X_train (the texts we’ll show the SVM as learning examples) and X_test (the texts we won’t show to the SVM so we can see if it’s able to predict the correct authors for these texts based on the patterns it learned from the texts in X_train). We’ll similarly break our labels (the author ids) into two arrays as well: y_train and y_test. We can use the function provided by scikit-learn to handle taking a random sample of our texts and labels, while making sure that the indices still correspond, as follows.

In [ ]:
# We use a fixed random_state to ensure that the same random sampling is used every time the code is run.
 
X_train, X_test, y_train, y_test = train_test_split(vectors, authors, test_size=0.2, random_state=1337)


We now have 40000 texts (80% of our data) to train on and 10000 (20%) for testing:

In [ ]:
X_train.shape, X_test.shape

### TRAINING AND TESTING A CLASSIFIER

We first need to call fit on our classifier and pass in the learning texts and labels. Then we’ll call predict to get predictions on the test data, and look at some metrics to see how well it did.

In [ ]:
%%time
svm = LinearSVC()
svm.fit(X_train, y_train)

We can now make predictions on the test set (note that the SVM has never seen the labels from the test set that are stored in y_test). The SVM will output whichever user_id it thinks is most likely to be the author of that review, for each review we pass in.

In [ ]:
predictions = svm.predict(X_test)

In [ ]:
correct_prediction = 0
for test, prediction in zip(predictions, y_test):
    if prediction == test:
        correct_prediction += 1

If the classifier did a good job, the predictions should be similar to the test labels (y_test).

In [ ]:
correct_prediction/len(y_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, predictions)

### IMPROVING OUR SYSTEM

Let’s see if we can tune some parameters and do even better.

The following vectorizer looks at words individually (unigrams) but also looks at pairs of words (bigrams). This makes our feature space much larger, so we’ll need a bit more processing time, both to create the vectors, and to train the SVM using the vectors. To alleviate this issue, we’ll tell the vectorizer to ignore all words and word pairs that don’t appear in at least five different reviews — there are a lot of very rarely used words, and we can’t learn anything from these anyway.

In [ ]:
%%time
vectorizer = TfidfVectorizer(ngram_range=(1,2), min_df=5)
vectors = vectorizer.fit_transform(texts)

In [ ]:
print(f"Now we have {vectors.shape[1]} features, about double the number we had before.\n\nWe can create our train test split again, and run the fitting and prediction code as before.")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(vectors, authors, test_size=0.2, random_state=1337)

In [ ]:
%%time
svm = LinearSVC()
svm.fit(X_train, y_train)
predictions = svm.predict(X_test)

In [ ]:
print(f"We can now identify the correct author correctly {accuracy_score(y_test, predictions)*100}% of the time.\nConsidering that some of the reviews are only a few sentences long, it is perhaps surprising that the writing styles are inactive enough.")

### CONCLUSION

In this post, you learnt how to use Python and scikit-learn for authorship identification. You can now prepare text data and train a simple classifier. You know a bit about the vectorization process, and the choices that have to be made regarding data preparation.

It might seem useless to predict labels that we already have, but we could use this same principle for many practical tasks. For example, it could help find people who have more than one Yelp account for the purposes of promoting their own establishment or leaving bad reviews for competitors. It is also useful in forensic linguistics when the true authorship of someone’s will or suicide note is often questioned, and it can be used to prove the authorship of disputed literary works, such as Shakespeare’s plays or books written under pseudonyms.